In [7]:
import numpy as np
import numpy as np
from PIL import Image
import os
import pickle

In [8]:
def sigmoid(x):
    return 1.0/(1.0 + np.exp(-x))
 
def sigmoid_derivada(x):
    return sigmoid(x)*(1.0-sigmoid(x))
 
def tanh(x):
    return np.tanh(x)
 
def tanh_derivada(x):
    return 1.0 - x**2
 
class RedNeuronal:
 
    def __init__(self, layers, activation='tanh'):
        if activation == 'sigmoid':
            self.activation = sigmoid
            self.activation_prime = sigmoid_derivada
        elif activation == 'tanh':
            self.activation = tanh
            self.activation_prime = tanh_derivada
 
        # inicializo los pesos
        self.weights = []
        self.deltas = []
        for i in range(1, len(layers) - 1):
            r = 2*np.random.random((layers[i-1] + 1, layers[i] + 1)) -1
            self.weights.append(r)
        r = 2*np.random.random( (layers[i] + 1, layers[i+1])) - 1
        self.weights.append(r)
 #entrenar
    def entrenar(self, X, y, learning_rate=0.2, epochs=100000):

        ones = np.atleast_2d(np.ones(X.shape[0]))
        X = np.concatenate((ones.T, X), axis=1)
        
        for k in range(epochs):
            i = np.random.randint(X.shape[0])
            a = [X[i]]
            
#forward propagation
            
            for l in range(len(self.weights)):
                    dot_value = np.dot(a[l], self.weights[l])
                    activation = self.activation(dot_value)
                    a.append(activation)

            error = y[i] - a[-1]
            deltas = [error * self.activation_prime(a[-1])]

            for l in range(len(a) - 2, 0, -1): 
                deltas.append(deltas[-1].dot(self.weights[l].T)*self.activation_prime(a[l]))
            self.deltas.append(deltas)
 

            deltas.reverse()
 
#backpropagation
            
            for i in range(len(self.weights)):
                layer = np.atleast_2d(a[i])
                delta = np.atleast_2d(deltas[i])
                self.weights[i] += learning_rate * layer.T.dot(delta)
 
            if k % 100 == 0:
                mse = np.mean(np.square(error))
                print(f"épocas: {k}  |  error: {mse}")

    def predict(self, x):
        ones = np.atleast_2d(np.ones(x.shape[0]))
        a = np.concatenate((np.ones(1).T, np.array(x)), axis=0)
        for l in range(0, len(self.weights)):
            a = self.activation(np.dot(a, self.weights[l]))
        return a



In [9]:
#cargo imagen y reduzco 100x100, la escalo a grises y aplano
def cargar_imagen(ruta, tamano=(100, 100)):
    img = Image.open(ruta).convert('L').resize(tamano)
    return np.array(img).flatten() / 255.0
# cargo la carpeta y asigno labels 
def cargar_carpeta(directorio, tamano=(100, 100)):
    X = []
    y = []
    for etiqueta, clase in enumerate(["otros", "yo"]):
        clase_dir = os.path.join(directorio, clase)
        if not os.path.isdir(clase_dir):
            continue
        for archivo in os.listdir(clase_dir):
            ruta = os.path.join(clase_dir, archivo)
            try:
                vector = cargar_imagen(ruta, tamano)
                X.append(vector)
                y.append(etiqueta)
            except:
                print(f"Error cargando la imagen: {ruta}")
    return np.array(X), np.array(y)


In [19]:
#llamo para asignar xs y ys
#le asigno la Ruta de la carpeta donde se tienen que encontrar si o si
# las carpetas de nombre Yo y otra carpeta de nombre otros
#en yo solo habran fotos de mi cara y en otros habran fotos de varia gente
X, y = cargar_carpeta(r"C:\Users\Kendrita\Desktop\perceptron\datos")

# Red neuronal: 400 entradas, 16 neuronas ocultas, 1 salida
#red = RedNeuronal(entradas=10000, ocultas=80, salidas=1)

# llamo la red enuronal le asigno 100000 de entrada, 512 neuronas de capas medias y 128 capas y 1 salida
red= RedNeuronal([10000, 512,128, 1],activation='sigmoid')
red.entrenar(X,y,learning_rate=0.05, epochs=1000)
#red.entrenar(X, y, epochs=1000, lr=0.5)


épocas: 0  |  error: 0.0030611182736165494
épocas: 100  |  error: 0.0034692883139333056
épocas: 200  |  error: 0.011046449714477142
épocas: 300  |  error: 0.03374182924456096
épocas: 400  |  error: 0.9686655171188426
épocas: 500  |  error: 0.011014297554274759
épocas: 600  |  error: 0.26062319874233025
épocas: 700  |  error: 4.335214503971692e-07
épocas: 800  |  error: 0.009953084163816328
épocas: 900  |  error: 0.0039166465951750344


In [24]:

#pongo la ruta de la imagen que quiero que clasifique
ruta = r"C:\Users\Kendrita\Desktop\perceptron\test\imagen_prueba8.jpeg"
vector = cargar_imagen(ruta)

#probabilidad = red.forward(np.array([vector]))[0][0]
probabilidad = float(red.predict(vector))


umbral = 0.8
if probabilidad >= umbral:
    print(f"00000 Soy yo 00000 (probabilidad: {probabilidad*100:.2f}%)")
elif probabilidad <0.8 and probabilidad >= 0.5:
    print(f"00000 Puedo ser yo 00000 (probabilidad: {probabilidad*100:.2f}%)")
else:
    print(f"XXXXX no soy yo XXXXX (probabilidad: {probabilidad*100:.2f}%)")

00000 Soy yo 00000 (probabilidad: 99.53%)


C:\Users\Kendrita\AppData\Local\Temp\ipykernel_4440\2141404201.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  probabilidad = float(red.predict(vector))
